Script for performing SOM training and saving results.

In [ ]:
from nextsomcore.nextsomcore import NxtSomCore
import pickle

Specify parameter for SOM

In [3]:
import argsSOM

arg = argsSOM.Args()
#print(arg.kmeans_max)


25


In [4]:
#arg.input_file="D:\\advangeo\\ghanaNW\\05_ParamModels\\Prediction\\PM_012\\Workspace\\SOM.lrn"
arg.input_file="nextsomcore/data/input/SOM.lrn"

arg.output_folder="nextsomcore/data/output"         # Folder to save som dictionary and cluster dictionary
arg.output_file_somspace="nextsomcore/data/output/somspace.txt"   
arg.outgeofile="nextsomcore/data/output/geospace.txt"   

# If input data is geotiff: list geotiff files, separated by "," ["name1.tiff","name2.tiff"]
#input_list_text=[]
#input_file= ",".join(input_list_text)
        
arg.som_x=10                # X dimension of generated SOM
arg.som_y=10                # Y dimension of generated SOM
arg.epochs=10               # Number of epochs to run

arg.maptype='toroid'            # Type of SOM (sheet, toroid)
arg.initialcodebook=None        # File path of initial codebook, 2D numpy.array of float32.
arg.neighborhood='gaussian'     # Shape of the neighborhood function. gaussian or bubble
arg.std_coeff=0.5               # Coefficient in the Gaussian neighborhood function
arg.radius0=0                   # Initial size of the neighborhood
arg.radiusN=1                   # Final size of the neighborhood
arg.radiuscooling='linear'      # Function that defines the decrease in the neighborhood size as the training proceeds (linear, exponential)
arg.scalecooling='linear'       # Function that defines the decrease in the learning scale as the training proceeds (linear, exponential)
arg.scale0=0.1                  # Initial learning rate
arg.scaleN=0.01                 # Final learning rate
arg.initialization='random'     # Type of SOM initialization (random, pca)
arg.gridtype='rectangular'      # Type of SOM grid (hexagonal, rectangular)
#arg.xmlfile="none"              # SOM inputs as an xml file

arg.kmeans="true"           # Run k-means clustering
arg.kmeans_init=5           # Number of initializations
arg.kmeans_min=2            # Minimum number of k-mean clusters
arg.kmeans_max=25           # Maximum number of k-mean clusters

#arg.output_file_geospace=None   # Text file that will contain calculated values: {X Y Z} data1 data2 dataN som_x som_y cluster b_data1 b_data2 b_dataN in geospace.
arg.geotiff_input=None      # geotiff_input
arg.normalized="false"      # Whether the data has been normalized or not
arg.minN=0                  # Minimum value for normalization
arg.maxN=1                  # Maximum value for normalization
arg.label=None              # Whether data contains label column, true or false


Run SOM training and save the results. Uses NxtSomCore package to do the actual work.

In [11]:
import functions.do_nextsomcore_save_results as dnsr

dnsr.run_SOM(arg)

AttributeError: module 'functions.do_nextsomcore_save_results' has no attribute 'run_SOM'

In [ ]:
nxtsomcore = NxtSomCore()
if initialcodebook is not None: #if initial codebook was provided (in the form of som.dictionary), open the file, and load som codebook from it.
    with open(initialcodebook, 'rb') as som_dictionary_file:
        som_dictionary = pickle.load(som_dictionary_file)
        initialcodebook=som_dictionary['codebook']
        initialization=None           
header = nxtsomcore.load_data(input_file) 
som = nxtsomcore.train(
    header['data'],
    som_x,
    som_y,
    epochs,
    kerneltype=0,
    verbose=1,
    neighborhood=neighborhood,
    std_coeff=std_coeff,
    maptype=maptype,
    radiuscooling=radiuscooling,
    scalecooling=scalecooling,
    initialization=initialization,
    initialcodebook=initialcodebook,
    gridtype=gridtype
    )          
if(output_folder==""):
    output_folder="C:/Temp/NextSom"
else:
    output_folder=output_folder
print(output_folder)
if(kmeans=="true"):
    som['clusters']=nxtsomcore.clusters(som,kmeans_min,kmeans_max,kmeans_init,output_folder)     

if outgeofile is not None:
    nxtsomcore.save_geospace_result(outgeofile, header, som, output_folder, input_file, normalized, label) 

nxtsomcore.save_somspace_result(output_file_somspace, header, som, output_folder, input_file, normalized)  
if(geotiff_input is not None):
    inputFileArray=geotiff_input.split(",")    
    nxtsomcore.write_geotiff_out(output_folder, inputFileArray[0])
with open(output_folder+'/som.dictionary', 'wb') as som_dictionary_file:
    pickle.dump(som, som_dictionary_file) #save som object to file.